# Задание 1

Сначала проведём в данном интеграле замену: $x(t) = a+\dfrac{b-a}{2}(1-\cos{\pi t})=[a=-1, b=1]=-\cos{\pi t}$

$$I = \int_{-1}^1\dfrac{\sin{\dfrac{x}{2}}}{e^x-1}dx = -\pi\int_0^1\dfrac{\sin{\left(\dfrac{\cos{\pi t}}{2}\right)}}{e^{-\cos{\pi t}}-1}\sin{\pi t}dt$$

Теперь подинтегральная функция регулярна на всём отрезке интегрирования и позволяет выходить за его пределы

In [ ]:
import numpy as np
from scipy.special import comb
from math import factorial

def D(n, j, Ds):
    
    if j+1==0:
        return 1
    else:
        s = 0
        for k in range(0,n+2*j+1):
            for l in range(0,j+1):
                s+=(-1)**k*Ds[l][n]*comb(n+2*l, k-j+l)*(n+2*j-2*k)**(n+2*j+2)/2**(n+2*j+2)/factorial(n+2*j+2)
        return s
    
def A(m,k,n, Ds):
    s=0
    
    for l in range(0,m+1):
        s+=(-1)**(k-m)* Ds[l][n] * comb(n+2*l, k-m+l)
    return s

def W(k,m, Ds):
    s=0
    for n in range(0,m+1):
        s+=A(m-n,k,2*n, Ds)/2**(2*n)/factorial(2*n+1)
    return s

def integr(f,h,m, b, a):
    s=0
    xs = np.linspace(a-h*m,b+h*m, int(2*m+(b-a)/h+1))
    ys = f(xs)
    J = int((b-a)/h)


    # Сохраняем значения коэффициентов D_n^l, чтобы не пересчитывать каждый раз 
    # Это было бы очень затратно
    Ds = np.ones(shape=(2*m+1,4*m+1))
    for l in range(-1,2*m):
        for n in range(0,4*m):
            Ds[l+1][n] = D(n,l, Ds)
            
    for j in range(0, J):
        for k in range(-m, m+1):
            s+=W(m, m-k, Ds)*ys[j+k+m]

    return h*s

In [137]:
def f(x):
    return np.sin(np.cos(np.pi*x)/2)*np.sin(np.pi*x)/(np.exp(-np.cos(np.pi*x))-1)

I = -np.pi*integr(f,0.0001, 10, 1,0)

dI = abs(-np.pi*integr(f,0.0002, 10, 1,0)-I)/(2**(2*10+2)-1)

I,dI

(1.0130255856983468, 3.02409744182496e-11)

По методу Рунге оценим погрешность, и получим что она всё ещё заметно больше, чем машинная точность, а при этом шаг интегрирования уже довольно мал (дальнейшее уменьшение приводит к значительному росту времени вычислений), а число шагов за пределы интегрирования больше, чем было рекомендовано для машинной точности ($m\approx 7$), и дальнейшее его увеличение также сильно снижает скорость вычислений.

Также заметим, что значение интеграла отличается от посчитанного на Вольфраме ($1.01304$) уже в 5 знаке после запятой, то есть оценка погрешности по Рунге, видимо, неверна, то есть не были выполнены условия применимости этой формулы.

# Задание 2

Сначала преобразуем предел суммы к частичной сумме ряда: $$1+\dfrac{1}{2}+\cdots+\dfrac{1}{n}-\ln{n} = \sum_{k=1}^{n}\left(\dfrac{1}{k}-\ln{k}\right)+\sum_{k=1}^{n-1}\ln{k} = 1 + \sum_{k=1}^{n-1}\left(\dfrac{1}{k+1}+\ln{\dfrac{k}{k+1}}\right)$$

Тогда при взятии предела в изначальном выражении получим ряд: $$1+\sum_{k=1}^{\infty}\left(\dfrac{1}{k+1}+\ln{\dfrac{k}{k+1}}\right) = 1+ \int_{j_m+1/2}^{\infty}\left(\dfrac{1}{x+1}+\ln{\dfrac{x}{x+1}}\right)dx+\sum_{j=1}^{j_m-m}\left(\dfrac{1}{j+1}+\ln{\dfrac{j}{j+1}}\right) + $$ $$+\sum_{j=j_m-m+1}^{j_m+m}(1-I^m_{j+m-j_m-1})\left(\dfrac{1}{j+1}+\ln{\dfrac{j}{j+1}}\right) + O(\dfrac{1}{j_s^{2m+2}})$$

Интеграл вычисляется точно: $$\int_{j_m+1/2}^{\infty}\left(\dfrac{1}{x+1}+\ln{\dfrac{x}{x+1}}\right)dx = -\left(j_m+\dfrac{1}{2}\right)\ln{\dfrac{j_m+\dfrac{1}{2}}{j_m+\dfrac{3}{2}}}-1$$

Так как хотим характерный масштаб изменения функции (например изменение в 2 раза) $j_s \gg 1$ при $j>j_m$, то можем взять, например, $j_m = 100$, $j_s=25$, также возьмём $m=15$

In [ ]:
j_m=100
j_s=25
m=15

Ds = np.ones(shape=(2*m+1,4*m+1))
for l in range(-1,2*m):
    for n in range(0,4*m):
        Ds[l+1][n] = D(n,l, Ds)

def I(l,m):
    s=0
    for k in range(0, l+1):
        s+=W(2*m-k, m, Ds)
    return s

I_1 = -(j_m+1/2)*np.log((j_m+1/2)/(j_m+3/2))

S_1 = np.sum([1/(j+1)+np.log(j/(j+1)) for j in range(1,j_m-m+1)])

S_2 = np.sum([(1/(j+1)+np.log(j/(j+1)))*(1-I(j+m-j_m-1,m)) for j in range(j_m-m+1,j_m+m)])

Int = I_1+S_1+S_2

Int, 1/j_s**(2*m+2)

(0.5772156649015318, 1.844674407370955e-45)

Видим, что несмотря на чрезвычайно низкую погрешность, посчитанную по формуле $\dfrac{1}{j_s^{2m+2}}$, расхождение с табличным значением константы наблюдается уже в 14 знаке после запятой (так что формально мы получили ответ с требуемой точностью - успех).

# Задание 3

В разобранном примере мы получили формулу для значения суммы исходного ряда через новый ряд $$S = \dfrac{\pi^2}{6}-\dfrac{\pi}{6}+1-\sum_{n=1}^{\infty}\dfrac{\cos{2n}}{n^2(n^4+n^2+1)}$$ и примерное число слагаемых для достижения требуемой точности ($k\approx 10$)

Осталось реализовать:

In [138]:
S_1 = 0

for n in range(1,15): #Возьмём чуть больше членов, чтобы гарантированно превысить точность
    S_1+=np.cos(2*n)/(n**2*(n**4+n**2+1))

S = np.pi**2/6-np.pi/6+1-S_1

S

2.266728526716446

# Задание 4

Ввиду того, что заданная функция кусочна, симметризовать сетку не получится, поэтому можем вычислить 2 односторонних предела по известной формуле и сравнить их: если они совпадут - мы нашли искомый предел. $$\lim\limits_{x\to +0}\dfrac{\ch{\sqrt{|x|}}-1}{x} = \lim\limits_{t\to 0} \dfrac{\ch{|t|}-1}{t^2}$$ 

Полученная функция чётная, что позволяет использовать нужный метод

In [154]:
h = 0.001
m=30
N=2*m

def f1(t):
    return (np.cosh(np.abs(t))-1)/t**2

def lim(ys, N, m):
    s=0
    for n in range(0,N+1):
        for k in range(0,2*m + (n%2)+1):
            s+=A(int(m-(n-(n%2))/2), k,n,Ds)*(-1)**n/4**n/factorial(n)*ys[abs(2*m+(n%2)-2*k)]

    return s

In [ ]:
xs = np.linspace(h, h*(2*m+2), 2*m+2)
ys = f1(xs)

Ds = np.ones(shape=(m+1,N+1))
for l in range(-1,m):
    for n in range(0,N):
        Ds[l+1][n] = D(n,l, Ds)

lim(ys,N,m)

0.5000000659956649

Теперь делаем то же самое с другим куском функции:

In [157]:
def f2(t):
    return -(np.cos(np.abs(t))-1)/t**2

ys = f2(xs)

lim(ys,N,m)

0.4999999339306714

Действительно односторонние пределы совпали с хорошей точностью (можно подгонять под требуемую точность путём увеличения порядка разложения $N$ или числа точек $m$) и равны $\approx \dfrac{1}{2}$.

Аналитическим вычислением предела можно убедиться, что предел действительно будет равен $\dfrac{1}{2}$